In [1]:
%autosave 0

Autosave disabled


In [49]:
import subprocess
from os import listdir
from os.path import isfile, join
from time import time, sleep
from binascii import hexlify

In [35]:
import tagcore

# Tagfuse File System Feature Test

This file contains a set of tests that verify the features of the Tag Fuse Filesystem.

## Prerequisites

- Raspberry Pi with latest software (See ```01_RPi_Build.md``` for details on contructing and installing a new RPi Disk Image). Make sure that the lates TagNet Basestation Software is installed, too.
- Tag with latest software (see ```?``` for details on constructing and installing a new Tag Image)

### Verified Features
- Polling for tags
    - tested with two
- Dblk read
    - using ```tagdump```
- Dlbk special files
    - sizes reflect offsets of underlying record status

### To be Tested
- Panic verification
    - Panic byte file read is working
    - need to figure out if/how to erase
- GPS XYZ value
- GPS cmd
    - has been used extensively, just need to add test
- Poll count verification
- Reboot using sys/{Active,Golden,Nib,Backup,Running}

### Failed Tests

- getattr
    - rtc
    - panic/{count, byte}
    - test/{sum,zeros,ones}
    - sys/{golden,nib}

### Missing Features
- Image filesize and timestamp metadata
- Not all file timestamps are being set
- Poll event identifiers
- Date on GPS (designates acquisition time)

## Initial State

The following variables provide user environment control.

In [4]:
nid='1fbcd99fd29f'
#nid='658bc8e5205c'
#nid='a0b696b4fb29'

In [14]:
TREE_BASE       = '/home/pi/tags/'
TAG_BASE        = TREE_BASE + nid + '/tag'
TAG_SYS_BASE    = TAG_BASE + '/sys'
TAG_IMAGE_STORE = TAG_BASE + '/sd/0/img'
SPARSE_STORE    = '/home/pi/Desktop'
BS_IMAGE_STORE  = '/mnt/neptune/tagbin/test'

## NOTES / ISSUES
- writing image when no room available gets stuck in a loop - very bad

## Show list of Images stored on the Basestation

In [18]:
ls -al $BS_IMAGE_STORE

total 312
drwxrwxrwx 2 pi devgrp      0 Nov 16 15:21 ./
drwxrwxrwx 2 pi devgrp      0 Nov 16 15:15 ../
-rwxrwxrwx 1 pi devgrp 102484 Nov 16 15:19 0.4.303*
-rwxrwxrwx 1 pi devgrp 102484 Nov 16 15:20 0.4.304*
-rwxrwxrwx 1 pi devgrp 102484 Nov 16 15:21 0.4.305*


In [23]:
cmd='ls ' + BS_IMAGE_STORE + '/*'
basestation_image_list= [ f for f in subprocess.check_output(cmd, shell=True).split('\n') if f]
basestation_image_list

['/mnt/neptune/tagbin/test/0.4.303',
 '/mnt/neptune/tagbin/test/0.4.304',
 '/mnt/neptune/tagbin/test/0.4.305']

## Show Images on Tag

In [19]:
ls -al $TAG_IMAGE_STORE

total 0
drwxr-x--x 4 pi pi 0 Dec 31  1969 ./
drwxr-x--x 6 pi pi 0 Dec 31  1969 ../
-rw-rw-r-- 1 pi pi 0 Nov 16 15:35 0.4.270
-rw-rw-r-- 1 pi pi 0 Nov 16 15:35 0.4.274


## Show the state of software currently installed on the tag

In [20]:
!tree $TAG_SYS_BASE

/home/pi/tags/1fbcd99fd29f/tag/sys
├── active
│   └── 0.4.270
├── backup
├── golden
│   └── 0.4.302
├── nib
│   └── 0.4.270
├── rtc
└── running
    └── 0.4.302

5 directories, 5 files


## Exercise the Image Storage and Activation Logic

Perform the following steps:
1. Get Tag's currently active version and save it as original_vers
2. Get list of available images to load (stored on basestation)
3. Remove all images from Tag (should't be able to remove active image)
4. Load three of the available images
5. Set first of the newly loaded images as Active
5. Set second of the newly loaded images as Backup
5. Set third of the newly loaded images as Active
6. Set orignal_vers image as Active

### 1. Get currently active version

In [25]:
cmd='ls '+TAG_SYS_BASE+'/active'
original_vers=subprocess.check_output(cmd, shell=True)[:-1]
original_vers

'0.4.270'

### 2. Get list of available images to load (stored on basestation)

In [26]:
basestation_image_list = [f for f in listdir(BS_IMAGE_STORE) if isfile(join(BS_IMAGE_STORE, f))
                          and not f.startswith('.') and not f == original_vers]
basestation_image_list

['0.4.303', '0.4.304', '0.4.305']

### 3. Remove all images from Tag
Note: should't be able to remove active image.

In [27]:
!rm $TAG_IMAGE_STORE/*
!ls -l $TAG_IMAGE_STORE

total 0
-rw-rw-r-- 1 pi pi 0 Nov 16 15:38 0.4.270


### 4. Load three of the available images

Use `cp` to copy one of the software images in the IMAGE_STORE to the tag
Monitor progress using pv

In [28]:
import os
import struct as pystruct
IMAGE_INFO_SIG = 0x33275401
IMAGE_META_OFFSET = 0x140
IMAGE_INFO_DEFAULT = [IMAGE_INFO_SIG, 0x20000, (0x140*2)+0x1c, 
                      9999, 2, 0, 
                      0, 
                      99, 77, 
                      '\00'  * 10]
IMAGE_INFO_LEN = 2
#
# Struct created for accessing image info (little indian)
# sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major, main_tree, aux_tree, build_time, im_rev, im_model = image_info
#
IMB_FIELDS = '<LLLHBBLBB10s'
image_info_struct = pystruct.Struct(IMB_FIELDS)
IMAGE_MIN_SIZE  =  (IMAGE_META_OFFSET + image_info_struct.size)

In [59]:
def info_check(filename):
    image_info_obj = tagcore.core_headers.obj_image_info()
    with open(filename, 'rb') as infile:
        if not infile:
            return (NONE, NONE, NONE)
        infile.seek(0, 2) # seek to the end
        file_size = infile.tell()
        if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
        infile.seek(0, 0)    # seek to the beginnnig
        # get image info from input file and sanity check
        infile.seek(IMAGE_META_OFFSET) # seek to location of image info
        buf = infile.read(len(image_info_obj))
        print(len(buf), hexlify(buf))
        image_info = image_info_obj.set(buf)
        print(image_info)
#        print("file information")
#        sig, image_start, imagelength, im_build, im_minor, im_major, image_chk, \
#             im_rev, im_model, pad = image_info
#        pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, image_chk: 0x{:x}"
#        print(pstr.format(sig, image_start, imagelength, image_chk))
#        pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
#        print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
        if image_info['basic']['sig'] != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    return ((im_major, im_minor, im_build), imagelength)

In [60]:
for i in range(3 if len(basestation_image_list) >= 3 
               else len(basestation_image_list)):
    version = basestation_image_list[i]
    print(version)
    info_check(os.path.join(BS_IMAGE_STORE, version))

0.4.303
(34, '0154273300000200549001002f010400e8e4ae000101000000000000000000002a01')


error: unpack requires a string argument of length 1

In [30]:
for i in range(3 if len(basestation_image_list) >= 3 
               else len(basestation_image_list)):
    version = basestation_image_list[i]
    print(version)
    info_check(os.path.join(BS_IMAGE_STORE, version))
    !dd if=$BS_IMAGE_STORE/$version of=$TAG_IMAGE_STORE/$version status=progress

0.4.303
file information
  signature: 0x33275401, start: 0x20000, length: 0x19054, image_chk: 0xaee4e8
  version: (0.4.303(0x12f)), rev: 1, model: 1
dd: writing to '/home/pi/tags/1fbcd99fd29f/tag/sd/0/img/0.4.303': No such file or directory
1+0 records in
0+1 records out
435 bytes copied, 6.06016 s, 0.1 kB/s
0.4.304
file information
  signature: 0x33275401, start: 0x20000, length: 0x19054, image_chk: 0xaee4e8
  version: (0.4.303(0x12f)), rev: 1, model: 1
dd: writing to '/home/pi/tags/1fbcd99fd29f/tag/sd/0/img/0.4.304': No such file or directory
1+0 records in
0+0 records out
0 bytes copied, 1.62847 s, 0.0 kB/s
0.4.305
file information
  signature: 0x33275401, start: 0x20000, length: 0x19054, image_chk: 0xaee4e9
  version: (0.4.305(0x131)), rev: 1, model: 1
dd: writing to '/home/pi/tags/1fbcd99fd29f/tag/sd/0/img/0.4.305': No such file or directory
1+0 records in
0+0 records out
0 bytes copied, 3.97473 s, 0.0 kB/s


### 5. Set first of the newly loaded images as Active

Set the active version NOTE: REBOOTS TAG

In [ ]:
version=basestation_image_list[0]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/active/$version

In [ ]:
sleep(5)

In [ ]:
!ls -l $SYS_BASE/active

Try again with same version should fail

In [ ]:
version=basestation_image_list[2]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/active/$version

### 5. Set second of the newly loaded images as Backup

In [ ]:
!ls -l $SYS_BASE/backup

In [ ]:
version=basestation_image_list[1]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/backup/$version

In [ ]:
sleep(5)

In [ ]:
!ls -l $SYS_BASE/backup

### 5. Set third of the newly loaded images as Active

In [ ]:
version=basestation_image_list[2]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/active/$version

In [ ]:
sleep(5)

In [ ]:
!ls -l $SYS_BASE/active

### 6. Set orignal_vers image as Active

Set the backup version

In [ ]:
!ln $TAG_IMAGE_STORE/$original_vers $SYS_BASE/active/$original_vers

In [ ]:
!tree $SYS_BASE

In [ ]:
!ls -l $TAG_IMAGE_STORE

Reboot into the NIB

Reboot into Golden

## Stop the Fuse Driver

In [ ]:
STOP

In [ ]:
!fusermount -u ~/tags

In [37]:
help(tagcore)

Help on package tagcore:

NAME
    tagcore

FILE
    /mnt/neptune/mm/tools/utils/tagcore/tagcore/__init__.py

DESCRIPTION
    tagcore: utility and common routines for many tag things
    @author:   Eric B. Decker

PACKAGE CONTENTS
    base_objs
    core_emitters
    core_headers
    core_populate
    core_rev
    dt_defs
    gps_mon
    imageinfo
    influxtest
    json_emitters
    misc_utils
    net_emitters
    net_headers
    net_populate
    panic_headers
    sirf_defs
    sirf_emitters
    sirf_headers
    sirf_populate
    tagfile

SUBMODULES
    __init__

FUNCTIONS
    buf_str(buf)
        Convert buffer into its display bytes
    
    dump_buf(buf, pre='', desc='rec:  ')
    
    obj_dt_hdr()

DATA
    CORE_MINOR = 92
    CORE_REV = 21
    __all__ = ['CORE_REV', 'CORE_MINOR', 'buf_str', 'dump_buf', 'obj_dt_hd...
    __version__ = '0.4.5rc92'

VERSION
    0.4.5rc92




In [40]:
image_info_obj = tagcore.core_headers.obj_image_info()

In [44]:
len(image_info_obj)

34

In [41]:
help(image_info_obj)

Help on aggie in module tagcore.base_objs object:

class aggie(collections.OrderedDict)
 |  aggie: aggregation node.
 |  takes one parameter a dictionary of key -> {atom | aggie}
 |  
 |  Method resolution order:
 |      aggie
 |      collections.OrderedDict
 |      __builtin__.dict
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, a_dict)
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |  
 |  build(self)
 |  
 |  set(self, buf)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from collections.OrderedDict:
 |  
 |  __delitem__(self, key, dict_delitem=<slot wrapper '__delitem__' of 'dict' objects>)
 |      od.__delitem__(y) <==> del od[y]
 |  
 |  __eq__(self, other)
 |      od.__eq__(y) <==> od==y.  Comparison to another OD is order-sensitive
 |      while comparison to a regular mapping is order-insensitive.
 |  
 |  __iter__(self)
 |      od.__iter__() <==> iter(od)
 |  
 |  __ne__(self, other)
 |  